In [22]:
import sys, json, re, collections
import pandas as pd
from pathlib import Path
from decouple import config
sys.path.append("../src/")
from llm_helpers import openai_client

pd.set_option('display.max_columns', 1000, 'display.width', 1000, 'display.max_rows',1000)

data_dir = Path(".").absolute().parent/"data"
ls = lambda p:print("\n".join(map(str,p.iterdir())))

ls(data_dir)

/Users/ugoren/Code/PR/llm_workshop/data/sample_apps.parquet


In [2]:
df = pd.read_parquet(data_dir / "sample_apps.parquet").sample(9)
df

,bundle_id,title,description,store_url,category_names,ios
4263,359917414,Solitaire,Solitaire by MobilityWare is the ORIGINAL make...,https://apps.apple.com/us/app/solitaire/id3599...,"Games,Casino,Card",True
30750,com.lazada.android,Lazada - Online Shopping App!,Lazada boast a large selection of and assortme...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False
59980,net.peakgames.toonblast,Toon Blast,From the creators of Toy Blast comes the ultim...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False
45897,com.staplegames.blocksClassicSGGP,Block Puzzle - Classic Style,Block Puzzle - Classic Style\n\nHow to play:\n...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False
56472,in.playsimple.wordtrip,Word Trip,WINNER OF THE PRESTIGIOUS ACADEMICS' CHOICE MI...,https://play.google.com/store/apps/details?id=...,"GAME_WORD,GAME",False
38654,com.pandora.android,Pandora - Music & Podcasts,Pandora gives you a personalized listening exp...,https://play.google.com/store/apps/details?id=...,"MUSIC_AND_AUDIO,APPLICATION",False
19386,com.exoticmatch.game,Zen Match,Playing Zen Match for 10 minutes a day sharpen...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False
51356,com.walmart.android,Walmart Shopping & Grocery,Save money. Live better.\n\nThe Walmart app is...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False
6673,892521917,Tiki Solitaire TriPeaks,Tiki Solitaire TriPeaks: the classic Solitaire...,https://apps.apple.com/us/app/tiki-solitaire-t...,"Games,Card,Puzzle",True


## Asking OpenAI directly?

In [4]:
def openai_ask(prompts):
    response = openai_client.completions.create(
        model="text-davinci-003",
        prompt=prompts,
    )
    ret = [choice.text.strip().lower() for choice in response.choices]
    return ret

openai_ask("Is a dog a mammal?")

['yes, a dog is a mammal.']

In [19]:
openai_ask("Is a dog a mammal? answer yes or no")

['yes']

## What about "fuzzier" questions?

In [14]:
openai_ask("I'm facing financial troubles, should I take a loan?")

['the decision if you should take a loan is a personal one, and']

In [15]:
openai_ask("I'm facing financial troubles, should I take a loan? answer yes or no")

['no']

In [16]:
openai_ask("I'm facing financial troubles, should I take a loan? answer yes or no")

['no. taking out a loan should only be done with careful consideration.']

In [18]:
openai_ask("I'm facing financial troubles, should I take a loan? answer yes or no")

['no. before taking a loan, review your budget and all other options']

## Popular approach: Retry

In [26]:
def yes_no_or_retry(prompt):
    i=0
    answer = ""
    while answer not in {"yes", "no"}:
        answer = openai_ask(prompt)[0].lower()
        i+=1
    return i, answer=="yes"

yes_no_or_retry("I'm facing financial troubles, should I take a loan? answer yes or no")

(2, False)

As we can see, the answer is not always a valid choise "yes" or "no"
# Introducing verbalizers:

## Verbalizers in a sentence:

Mapping a **class** to a **token**, and then choosing the token with the highest logit
## The OpenAI tokenizer

https://platform.openai.com/tokenizer

In [20]:
YES_TOKEN = frozenset([5297, 3763, 3363, 8505, 3363, 3763, 43335, 3763, 21560])
GPT3_YES_TOKEN = frozenset([9642, 14410, 10035, 7566, 14331, 9891])
NO_TOKEN = frozenset([2949, 645, 1400, 3919, 1400, 645, 15285, 645, 8005])
GPT3_NO_TOKEN = frozenset([2822, 5782, 912, 2201, 9173, 2360])


def openai_yes_or_no(prompts):
    response = openai_client.completions.create(
        model="text-davinci-003",
        prompt=prompts,
        temperature=0,
        logit_bias={t: 100 for t in YES_TOKEN | NO_TOKEN},
        max_tokens=1,
    )
    ret = [choice.text.strip().lower() == "yes" for choice in response.choices]
    return ret

openai_yes_or_no("Is dog a mammal?")


[True]

In [21]:
openai_yes_or_no("I'm facing financial troubles, should I take a loan? answer yes or no")

[False]

In [23]:
prompt = "I'm facing financial troubles, should I take a loan? answer yes or no"
prompts = [prompt]*10
answers = openai_yes_or_no(prompts)
collections.Counter(answers)

Counter({False: 10})

## Question:
Ask the following questions for each app
1. Is the app for kids?
2. Is the app a shopping app?
3. Is it a game?
4. Is this app a dating app?
5. Does this app have in-app purchases?
6. Is this app a match 3 game?

Compare the naive "yes" or "no" to logit bias